In [8]:
from utils.util import load_json_line_by_line
from collections import Counter
data = load_json_line_by_line("./output\ECIhardprompt\generated_predictions (7).jsonl")
def get_label_from_str(text):
    return text.split("\n", 1)[0]    
def find_match_substring(text):
    if "none" in text:
        return 0
    elif "caused by" in text:
        return -1
    elif "cause" in text:
        return 1
    else:
        return 2
#label2index 1 -1 0
label2index = {"event1 and event2 have no causal relationship":0,"event1 caused by event2":-1,"event1 cause event2":1}
ground_truth = [label2index[d["label"]] for d in data]
predictions = [d["predict"] for d in data]
true_predictions = [get_label_from_str(p) for p in predictions]
predictions1 = [find_match_substring(p) for p in true_predictions]
print(Counter(predictions1))
print(Counter(ground_truth))
#get content before \n



Counter({0: 728, 1: 232, -1: 39, 2: 1})
Counter({0: 839, -1: 82, 1: 79})


In [19]:
from collections import defaultdict  
  
def calculate_per_class_metrics(ground_truth, predictions):  
    # 创建字典来存储每个类别的TP, FP, FN计数  
    tp_fp_fn = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})  
      
    # 遍历所有样本  
    for gt, pred in zip(ground_truth, predictions):  
        # 如果预测正确，增加TP计数  
        if gt == pred:  
            tp_fp_fn[gt]['TP'] += 1  
        # 如果预测错误但预测为正例，增加FP计数  
        elif pred != -1 and gt != pred:  # 假设-1是未分类或填充值，根据实际情况调整  
            tp_fp_fn[pred]['FP'] += 1  
        # 如果预测错误且实际为正例但预测为负例或未分类，增加FN计数  
        elif gt != -1 and gt != pred:  
            tp_fp_fn[gt]['FN'] += 1  
      
    # 计算每个类别的精确度、召回率和F1分数  
    metrics_per_class = {}  
    for label, counts in tp_fp_fn.items():  
        tp = counts['TP']  
        fp = counts['FP']  
        fn = counts['FN']  
          
        # 避免除以零的情况  
        precision = tp / (tp + fp) if tp + fp > 0 else 0  
        recall = tp / (tp + fn) if tp + fn > 0 else 0  
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0  
          
        metrics_per_class[label] = {  
            'Precision': precision,  
            'Recall': recall,  
            'F1 Score': f1_score  
        }  

    # 返回整体的P、R和F1         
    return metrics_per_class  
def calculate_accuracy(ground_truth, predictions):  
    # 使用列表推导式计算正确分类的样本数  
    correct_predictions = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == pred)  
      
    # 计算准确率  
    accuracy = correct_predictions / len(ground_truth)  
      
    # 返回准确率  
    return accuracy    

metrics = calculate_per_class_metrics(ground_truth, predictions1)  
for label, metric_values in metrics.items():  
    print(f"Label {label}: Precision={metric_values['Precision']:.2f}, Recall={metric_values['Recall']:.2f}, F1 Score={metric_values['F1 Score']:.2f}")
print(calculate_accuracy(ground_truth,predictions1))
print(Counter(ground_truth))


Label 0: Precision=0.86, Recall=0.96, F1 Score=0.91
Label 1: Precision=0.11, Recall=0.84, F1 Score=0.20
Label -1: Precision=1.00, Recall=1.00, F1 Score=1.00
Label 2: Precision=0.00, Recall=0.00, F1 Score=0.00
0.658
Counter({0: 839, -1: 82, 1: 79})


In [20]:
from utils.util import load_json_line_by_line
from collections import Counter
data = load_json_line_by_line("./output\ECIhardprompt\generated_predictions (9).jsonl")
def get_label_from_str(text):
    return text.split(".", 1)[0]    
def find_match_substring(text):
    if "no causal" in text:
        return 0
    elif "caused by" in text:
        return -1
    elif "cause" in text:
        return 1
    else:
        return 2
#label2index 1 -1 0
label2index = {"event1 and event2 have no causal relationship":0,"event1 caused by event2":-1,"event1 cause event2":1}
ground_truth = [label2index[d["label"]] for d in data]
predictions = [d["predict"] for d in data]
true_predictions = [get_label_from_str(p) for p in predictions]
predictions1 = [find_match_substring(p) for p in true_predictions]
print(Counter(predictions1))
print(Counter(ground_truth))
#get content before \n


Counter({0: 348, 2: 232, -1: 224, 1: 196})
Counter({0: 839, -1: 82, 1: 79})


In [21]:
metrics = calculate_per_class_metrics(ground_truth, predictions1)  
for label, metric_values in metrics.items():  
    print(f"Label {label}: Precision={metric_values['Precision']:.2f}, Recall={metric_values['Recall']:.2f}, F1 Score={metric_values['F1 Score']:.2f}")
print(calculate_accuracy(ground_truth,predictions1))
print(Counter(ground_truth))

Label 0: Precision=0.85, Recall=0.61, F1 Score=0.71
Label 1: Precision=0.11, Recall=0.60, F1 Score=0.18
Label 2: Precision=0.00, Recall=0.00, F1 Score=0.00
Label -1: Precision=1.00, Recall=1.00, F1 Score=1.00
0.339
Counter({0: 839, -1: 82, 1: 79})
